Goal of this notebook is to understand how to call cphf.solve_nos1(fvind, mo_energy, mo_occ, h1,max_cycle=20, tol=1e-9, hermi=False)  (better the shape of the arrays passed as arguments)

In [6]:
import pyscf
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
import matplotlib.pyplot as plt
from pyscf.geomopt.berny_solver import optimize
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
#from pyscf.scf._response_functions import _gen_rhf_response 

In [7]:
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [13]:
import time
from functools import reduce
import numpy
from pyscf import lib
from pyscf.lib import logger
from pyscf.scf import cphf
#from pyscf.soscf.newton_ah import _gen_rhf_response


def dipole(mf):
    return mf.dip_moment(mf.mol, mf.make_rdm1())


def polarizability(polobj, with_cphf=True):
    from pyscf.prop.nmr import rhf as rhf_nmr
    log = logger.new_logger(polobj)
    mf = polobj._scf
    mol = mf.mol
    mo_energy = mf.mo_energy
    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    occidx = mo_occ > 0
    orbo = mo_coeff[:, occidx]
    orbv = mo_coeff[:,~occidx]

    charges = mol.atom_charges()
    coords  = mol.atom_coords()
    charge_center = numpy.einsum('i,ix->x', charges, coords) / charges.sum()
    with mol.with_common_orig(charge_center):
        int_r = mol.intor_symmetric('int1e_r', comp=3)
    h1 = lib.einsum('xpq,pi,qj->xij', int_r, mo_coeff.conj(), orbo)
    print(h1.shape)
    print(mo_energy, mo_occ,h1)
    s1 = numpy.zeros_like(h1)
    vind = polobj.gen_vind(mf, mo_coeff, mo_occ)
    if with_cphf:
        mo1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1,
                         polobj.max_cycle_cphf, polobj.conv_tol,
                         verbose=log)[0]
    else:
        mo1 = rhf_nmr._solve_mo1_uncoupled(mo_energy, mo_occ, h1, s1)[0]

    e2 = numpy.einsum('xpi,ypi->xy', h1, mo1)
    # *-1 from the definition of dipole moment. *2 for double occupancy
    e2 = (e2 + e2.T) * -2
    return e2


In [20]:
mol = gto.M(atom='He 0 0 0; He 0 0 1.2', unit="Bohr",basis="3-21g")
mf = scf.RHF(mol)
e=mf.scf()

converged SCF energy = -4.99200502849037


In [21]:
from pyscf.prop.polarizability.rhf import Polarizability
plo=Polarizability(mf)

In [22]:
polarizability(plo)

(3, 4, 2)
[-1.50533059 -0.3966134   1.83544158  2.19029745] [2. 2. 0. 0.] [[[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 2.77555756e-17 -7.44003543e-01]
  [-7.44003543e-01 -1.22124533e-15]
  [-1.16573418e-15  3.90037923e-01]
  [ 2.18186488e-02  4.99600361e-16]]]


array([[-0.        , -0.        , -0.        ],
       [-0.        , -0.        , -0.        ],
       [-0.        , -0.        ,  0.34145845]])